In [1]:
import os

import tensorflow as tf
from transformers.modeling_tf_bert import TFBertPreTrainedModel, TFBertModel
from transformers import TFBertMainLayer, BertTokenizer
from transformers.modeling_tf_utils import get_initializer
import pandas as pd

In [2]:
data_dir = "google-quest-challenge/"

train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))
submit_df = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))

In [3]:
print(train_df.keys())
train_df.head()

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host', 'question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfa

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [4]:
test_df.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [5]:
submit_df.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [6]:
train_X_df = train_df[['question_title', 'question_body', 'answer', 'category']]
train_targets_df = train_df[["question_asker_intent_understanding",
    "question_body_critical",
    "question_conversational",
    "question_expect_short_answer",
    "question_fact_seeking",
    "question_has_commonly_accepted_answer",
    "question_interestingness_others",
    "question_interestingness_self",
    "question_multi_intent",
    "question_not_really_a_question",
    "question_opinion_seeking",
    "question_type_choice",
    "question_type_compare",
    "question_type_consequence",
    "question_type_definition",
    "question_type_entity",
    "question_type_instructions",
    "question_type_procedure",
    "question_type_reason_explanation",
    "question_type_spelling",
    "question_well_written",
    "answer_helpful",
    "answer_level_of_information",
    "answer_plausible",
    "answer_relevance",
    "answer_satisfaction",
    "answer_type_instructions",
    "answer_type_procedure",
    "answer_type_reason_explanation",
    "answer_well_written"]]

In [123]:
train_targets_df.head()

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [124]:
# We want to predict all needed answers in range 0 - 1 so 30 answers in range 0 -1 

num_labels = 30
import numpy as np

class BertForQALabeling(TFBertPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        
        self.num_layers = config.num_labels
        self.backbone = TFBertMainLayer(config, name="bert_backbone")
        
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(
            config.num_labels, kernel_initializer=get_initializer(config.initializer_range), name="classifier", activation='softmax')
        
    def call(self, inputs, **kwargs):
        
        bert_output = self.backbone(inputs, **kwargs)
        print(np.asarray(bert_output).shape)
        
        pooled_outputs = bert_output[1]
        
        print(np.asarray(pooled_outputs).shape)
        
        pooled_outputs = self.dropout(pooled_outputs, training=kwargs.get("training", False))
        logits = self.classifier(pooled_outputs)
        
        print("logits shape ", logits.shape)
        
        return logits
        

In [125]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [126]:
model = BertForQALabeling.from_pretrained('bert-base-uncased', num_labels=num_labels)

(2, 3)
(3, 768)
logits shape  (3, 30)
(2, 3)
(3, 768)
logits shape  (3, 30)


In [127]:
model.summary()

Model: "bert_for_qa_labeling_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert_backbone (TFBertMainLay multiple                  109482240 
_________________________________________________________________
dropout_151 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  23070     
Total params: 109,505,310
Trainable params: 109,505,310
Non-trainable params: 0
_________________________________________________________________


In [128]:
input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
print("inputssss ", type(input_ids))
outputs = model(input_ids)
print(outputs.shape)
logits = outputs[0]

inputssss  <class 'tensorflow.python.framework.ops.EagerTensor'>
(2, 1)
(1, 768)
logits shape  (1, 30)
(1, 30)


In [189]:
from math import ceil, floor

class dataPreprocessor(object):
    """
        Class for preprocessing input data so it will be in BERT format
    """
    logger = False
    tokenizer = None
    
    @classmethod
    def combine_data(cls, q_body, q_title, answer, max_seq_length):
        """
            Method for combining preprocessed data with tokens into form:
            
            [CLS]question_title[SEP]question_body[SEP]answer[SEP]
            
            Additionally it will padd data to max_seq_length
        """
        combined_data = [cls.tokenizer.cls_token_id] + \
                        q_title + \
                        [cls.tokenizer.sep_token_id] + \
                        q_body + \
                        [cls.tokenizer.sep_token_id] + \
                        answer + \
                        [cls.tokenizer.sep_token_id] \
                        
        combined_data = combined_data + [cls.tokenizer.pad_token_id]*(max_seq_length - len(combined_data))
        
        if cls.logger:
            print("combined data {}\n" .format(combined_data))
    
        return combined_data
    
    @classmethod
    def cut_data_lengths(cls, q_body, q_title, answer, max_seq_lengths):
        """
            Method that should cut each part of input data to fit max_seq_length 
            1.Encode data and convert from binary type to utf-8
            2. do preprocessing and head-tial adding
            3. combine data with [CLS] and [SEP]s
            4. return preprred data
        """
        q_title = q_title.numpy().decode("utf-8")
        q_body = q_body.numpy().decode("utf-8")
        answer = answer.numpy().decode("utf-8")
        
        q_title = cls.tokenizer.encode(q_title)
        q_body = cls.tokenizer.encode(q_body)
        answer = cls.tokenizer.encode(answer)
        """
        if cls.logger:
            print("sep token ", cls.tokenizer.sep_token_id)
            print("pad token ", cls.tokenizer.pad_token_id)
            print("cls token ", cls.tokenizer.cls_token_id)
            print("q_title {}\n\nq_body{}\n\nanswer{}\n\n" .format(q_title, q_body, answer))
            print("{}\n{}\n{}\n" .format(len(q_title), len(q_body), len(answer)))"""
        
        max_q_title_len, max_q_body_len, max_answer_len, max_seq_length = max_seq_lengths

        
        # we`re substracting 4 due to tokens addition after preprocessing
        # if length is ok just add tokens
        if len(q_title) + len(q_body) + len(answer) <= max_seq_length -4:
            combined_data = cls.combine_data(q_body=q_body, 
                                             q_title=q_title, 
                                             answer=answer,
                                             max_seq_length=max_seq_length)
            return combined_data
            
        # if question title is shorter than max length:
        # add difference divided bu two to max length of both question body and answer body
        if len(q_title) < max_q_title_len:
            diff = max_q_title_len - len(q_title)
            new_max_q_body_len = max_q_body_len + ceil((diff)/2)
            new_max_answer_len = max_answer_len + floor((diff)/2)
        
        # if question body length is shorter than max question body length:
        # add difference to max anser body length
        # else vice versa
        if len(q_body) < max_q_body_len:
            new_max_answer_len = max_answer_len + (max_q_body_len - len(q_body))
            new_max_q_body_len = len(q_body)
        elif len(answer) < max_answer_len:
            new_max_q_body_len = max_q_body_len + (max_answer_len - len(answer))
            new_max_answer_len = len(answer)
        else:
            new_max_answer_len = max_answer_len
            new_max_q_body_len = max_q_body_len
        
        # sanity check
        if (new_max_answer_len + new_max_q_body_len + max_q_title_len + 4) != max_seq_length:
            raise ValueError("Wrong sequence length  {} {} {} {} " \
                             .format(new_max_answer_len, new_max_q_body_len, max_q_title_len, max_seq_length))
        
        # paper : https://arxiv.org/pdf/1905.05583.pdf
        # head - Tail method
        # we`re taking input max number of elements as head
        # we`re taking remaining available elements as tail
        # we`re taking 1/3 as head 2/3 as tail
        head_q_body = round((new_max_q_body_len/3))
        tail_q_body = (new_max_q_body_len - head_q_body)
        
        head_answer = round((new_max_answer_len)/3)
        tail_answer = (new_max_answer_len - head_answer)
        
        head_q_title = max_q_title_len
        
        q_title = q_title[:head_q_title]
        q_body = q_body[:head_q_body] + q_body[-tail_q_body:]
        answer = answer[:head_answer] + answer[-tail_answer:]
        
        combined_data = cls.combine_data(q_body=q_body, 
                                         q_title=q_title, 
                                         answer=answer, 
                                         max_seq_length=max_seq_length)
        
        return combined_data
    
    @classmethod
    def preprocessBatch(cls, q_body, q_title, answer, max_seq_lengths):
        """
            Method for iterating through Batch in order to preprocess data
        """
        preprocessed_batch = None
        for i in range (q_body.shape[0]):
            preprocessed_element = tf.constant(cls.cut_data_lengths(q_body[i], q_title[i], answer[i], max_seq_lengths))[None,:]
            if preprocessed_batch == None:
                preprocessed_batch = preprocessed_element
            else:
                preprocessed_batch = tf.concat([preprocessed_batch, preprocessed_element], axis=0)
        print("final batch size = ", preprocessed_batch.shape)
        return preprocessed_batch

In [190]:
q_title = train_X_df['question_title'].values
q_body = train_X_df['question_body'].values
answer = train_X_df['answer'].values

In [191]:
dataPreprocessor.logger = False
dataPreprocessor.tokenizer = tokenizer

In [192]:
train_ds = tf.data.Dataset.from_tensor_slices((q_body, q_title, answer)).shuffle(len(q_body)//4, reshuffle_each_iteration=True).batch(batch_size=64, drop_remainder=True)

In [193]:
for _, (q_body, q_title, answer) in enumerate(train_ds):
    # do something here
    dataPreprocessor.preprocessBatch(q_body, q_title, answer, max_seq_lengths=(30,128,128, 290))
    break

Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1125 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (690 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (787 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

final batch size =  (64, 290)
